In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

import ipywidgets as widgets
from ipywidgets import interact

In [2]:
run 'utils/getFMext.py'

In [3]:
run 'utils/getMag.py'   

In [4]:
@interact(Av=(0, 5, .2), Rv=(2, 5, .5))
def f(Av, Rv):
    fig,[[ax1,ax2],[ax3,ax4]]=plt.subplots(2,2,figsize=(12,10))
    x1,f1=np.loadtxt('AATau_IR2008.txt',unpack=True)
    x2,f2=np.loadtxt('AATau_opt2008.txt',unpack=True)
    x3,f3=np.loadtxt('AATau_IR2014dec02.txt',unpack=True)
    x4,f4=np.loadtxt('AATau_opt2014dec02.txt',unpack=True)
    
    # mask out the NaNs:
    def maskNans(wavelength,data):
        masked_data=data[np.logical_not(np.isnan(data))]
        masked_wavelength=wavelength[np.logical_not(np.isnan(data))]
        return (masked_wavelength,masked_data)

    x1,f1=maskNans(x1,f1)
    x2,f2=maskNans(x2,f2)
    x3,f3=maskNans(x3,f3)
    x4,f4=maskNans(x4,f4)
    
    xobs1,fobs1=np.append(x2,x1)[1500:],np.append(f2,f1)[1500:]
    xobs2,fobs2=np.append(x4,x3)[500:],np.append(f4,f3)[500:]
    
    fobs1ext = fobs1 * 10.**(-0.4*Av*getFMext(xobs1,Rv,'fmunred')/Rv)
    
    ax1.plot(xobs1, fobs1, label='2008',color='blue')
    ax1.set_ylim(0,0.8E-13)
    ax1.set_xlabel('$\lambda$')
    ax1.set_ylabel('$F_\lambda$')    
    ax1.plot(xobs2, fobs2, label='2014',color='red')
    ax1.plot(xobs1, fobs1ext, label="ext'd 2008",color='black',alpha=0.6)
    ax1.legend(loc='best')

    ax2.plot(1./xobs1, fobs1, color='blue')
    ax2.set_ylim(0,0.8E-13)
    ax2.set_xlabel('1/$\lambda$')
    ax2.set_ylabel('$F_\lambda$')    
    ax2.plot(1./xobs2, fobs2, color='red')
    ax2.plot(1./xobs1, fobs1ext, color='black', alpha=0.6)
    
    J1=getMag('J', xobs1, 10.*fobs1)
    H1=getMag('H', xobs1, 10.*fobs1)
    Ks1=getMag('Ks', xobs1, 10.*fobs1)
    V1=getMag('V', xobs1, 10.*fobs1)
    R1=getMag('R', xobs1, 10.*fobs1)

    J2=getMag('J', xobs2, 10.*fobs2)
    H2=getMag('H', xobs2, 10.*fobs2)
    Ks2=getMag('Ks', xobs2, 10.*fobs2)
    V2=getMag('V', xobs2, 10.*fobs2)
    R2=getMag('R', xobs2, 10.*fobs2)

    J1ext=getMag('J', xobs1, 10.*fobs1ext)
    H1ext=getMag('H', xobs1, 10.*fobs1ext)
    Ks1ext=getMag('Ks', xobs1, 10.*fobs1ext)
    V1ext=getMag('V', xobs1, 10.*fobs1ext)
    R1ext=getMag('R', xobs1, 10.*fobs1ext)

    ax3.plot(H1-Ks1,J1-H1,'o',color='blue')
    ax3.set_xlabel('H-Ks')
    ax3.set_ylabel('J-H')
    ax3.plot(H2-Ks2,J2-H2,'o',color='red')
    ax3.plot(H1ext-Ks1ext,J1ext-H1ext,'o',color='black')
    ax3.plot([H1-Ks1,H2-Ks2],[J1-H1,J2-H2],color='red')
    ax3.plot([H1-Ks1,H1ext-Ks1ext],[J1-H1,J1ext-H1ext],color='black')
    
    ax4.plot(V1-R1,V1,'o',color='blue')
    ax4.set_xlabel('V-R')
    ax4.set_ylabel('V')
    ax4.plot(V2-R2,V2,'o',color='red')
    ax4.plot(V1ext-R1ext,V1ext,'o',color='black')
    ax4.plot([V1-R1,V2-R2],[V1,V2],color='red')
    ax4.plot([V1-R1,V1ext-R1ext],[V1,V1ext],color='black')

    plt.show()